In [1]:
!pip install mysql-connector-python

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np  # Import numpy for mean calculation
import mysql.connector as msc

conn = msc.connect(
    host="localhost",
    user="root",
    password="wasd",
    database="ir_policy"
)

# Midterm Performance eval

In [3]:
cursor1 = conn.cursor()
cursor1.execute("SELECT * FROM ir_policy.apps_data")
data1 = cursor1.fetchall() 
cursor1.close() 

cursor2 = conn.cursor()
cursor2.execute("SELECT * FROM ir_policy.generated_data_pegasus")
data2 = cursor2.fetchall()  
cursor2.close() 

True

In [4]:
privacy_policy_texts = [data[3] for data in data1] 
generated_data_texts = [data[1] for data in data2]  

print(privacy_policy_texts)

['Skip to content\nWhatsApp Main Page\n\nFeatures\nPrivacy\nHelp Center\nBlog\nWhatsApp Web\nDownload\nLast modified: January 4, 2021 (archived versions)\n\nWhatsApp Privacy Policy\nIf you live in the European Region, WhatsApp Ireland Limited provides the services to you under this Terms of Service and Privacy Policy.\n\nWhatsApp Legal Info\nIf you live outside the European Region, WhatsApp LLC ("WhatsApp," "our," "we," or "us") provides our Services to you under this Terms of Service and Privacy Policy.\n\nOur Privacy Policy ("Privacy Policy") helps explain our data practices, including the information we process to provide our Services.\n\nFor example, our Privacy Policy talks about what information we collect and how this affects you. It also explains the steps we take to protect your privacy, like building our Services so delivered messages aren’t stored by us and giving you control over who you communicate with on our Services.\n\nWe are one of the Meta Companies. You can learn mo

In [5]:
print(generated_data_texts)

['Skip to content WhatsAppFeatures Blog Stories For Business Who we are About us Careers Brand Center Privacy Use WhatsApp modified 2023 archived versions WhatsApp Features Blog Stories For Business Who we are About us Careers Brand Center Privacy Use WhatsApp If you live in Key Updates, WhatsApp provides the services to you under WhatsApp and Features Blog Stories For Business Who we are About us Careers Brand Center Privacy Use WhatsApp. Our Features Blog Stories For Business Who we are About us Careers Brand Center Privacy Use WhatsApp "Features Blog Stories For Business Who we are About us Careers Brand Center Privacy Use WhatsApp" helps explain our data practices, including the information we process to provide our Services.', 'You are a Acceptable Use Policy user if you have either registered by linking a mobile number that uses international dialing code 1313 "Child mobile number" ; or contracted with Shenzhen WeChat for Acceptable Use Policy. If you are a Acceptable Use Policy 

- Cosine Similarity

In [6]:
vectorizer = TfidfVectorizer()
generated_data_vectors = vectorizer.fit_transform(generated_data_texts)
privacy_policy_vectors = vectorizer.transform(privacy_policy_texts) 

cosine_similarities = cosine_similarity(generated_data_vectors, privacy_policy_vectors)

average_cosine_similarity = np.mean(cosine_similarities.diagonal())

for app_id, cosine_sim in zip([data[0] for data in data1], cosine_similarities.diagonal()):
    print(f"App ID {app_id}: Cosine Similarity = {cosine_sim:.2f}")

print(f"Average Cosine Similarity: {average_cosine_similarity:.2f}")

cursor1.close()
cursor2.close()
conn.close()

App ID 1: Cosine Similarity = 0.35
App ID 2: Cosine Similarity = 0.31
App ID 3: Cosine Similarity = 0.28
App ID 4: Cosine Similarity = 0.41
App ID 5: Cosine Similarity = 0.46
App ID 6: Cosine Similarity = 0.29
App ID 7: Cosine Similarity = 0.30
App ID 8: Cosine Similarity = 0.69
App ID 9: Cosine Similarity = 0.32
App ID 10: Cosine Similarity = 0.20
App ID 11: Cosine Similarity = 0.62
App ID 12: Cosine Similarity = 0.14
App ID 13: Cosine Similarity = 0.65
App ID 14: Cosine Similarity = 0.26
App ID 15: Cosine Similarity = 0.32
App ID 16: Cosine Similarity = 0.17
App ID 17: Cosine Similarity = 0.51
App ID 18: Cosine Similarity = 0.34
App ID 19: Cosine Similarity = 0.52
App ID 20: Cosine Similarity = 0.17
App ID 21: Cosine Similarity = 0.16
App ID 22: Cosine Similarity = 0.32
App ID 23: Cosine Similarity = 0.59
App ID 24: Cosine Similarity = 0.31
App ID 25: Cosine Similarity = 0.32
App ID 26: Cosine Similarity = 0.32
App ID 27: Cosine Similarity = 0.32
App ID 28: Cosine Similarity = 0.53
A

- ROUGE SCORES

In [7]:
from rouge import Rouge

rouge = Rouge()

aggregate_scores = {'rouge-1': {'f': [], 'p': [], 'r': []},
                    'rouge-2': {'f': [], 'p': [], 'r': []},
                    'rouge-l': {'f': [], 'p': [], 'r': []}}

for app_id, generated_text, reference_text in zip([data[0] for data in data1], generated_data_texts, privacy_policy_texts):
    scores = rouge.get_scores(generated_text, reference_text)[0]
    
    for key in aggregate_scores.keys():
        aggregate_scores[key]['f'].append(scores[key]['f'])
        aggregate_scores[key]['p'].append(scores[key]['p'])
        aggregate_scores[key]['r'].append(scores[key]['r'])
        
    print(f"App ID {app_id} ROUGE scores:")
    for rouge_key, rouge_scores in scores.items():
        print(f"{rouge_key}: f({rouge_scores['f']:.2f}), p({rouge_scores['p']:.2f}), r({rouge_scores['r']:.2f})")

print("Aggregate ROUGE scores:")
for rouge_key, score_lists in aggregate_scores.items():
    f_mean = np.mean(score_lists['f'])
    p_mean = np.mean(score_lists['p'])
    r_mean = np.mean(score_lists['r'])
    print(f"{rouge_key}: f({f_mean:.2f}), p({p_mean:.2f}), r({r_mean:.2f})")

App ID 1 ROUGE scores:
rouge-1: f(0.08), p(0.85), r(0.04)
rouge-2: f(0.03), p(0.63), r(0.01)
rouge-l: f(0.07), p(0.83), r(0.04)
App ID 2 ROUGE scores:
rouge-1: f(0.06), p(0.90), r(0.03)
rouge-2: f(0.02), p(0.68), r(0.01)
rouge-l: f(0.06), p(0.88), r(0.03)
App ID 3 ROUGE scores:
rouge-1: f(0.11), p(0.97), r(0.06)
rouge-2: f(0.04), p(0.78), r(0.02)
rouge-l: f(0.11), p(0.97), r(0.06)
App ID 4 ROUGE scores:
rouge-1: f(0.13), p(0.97), r(0.07)
rouge-2: f(0.06), p(0.87), r(0.03)
rouge-l: f(0.13), p(0.97), r(0.07)
App ID 5 ROUGE scores:
rouge-1: f(0.12), p(1.00), r(0.06)
rouge-2: f(0.06), p(0.93), r(0.03)
rouge-l: f(0.12), p(1.00), r(0.06)
App ID 6 ROUGE scores:
rouge-1: f(0.07), p(1.00), r(0.04)
rouge-2: f(0.02), p(0.67), r(0.01)
rouge-l: f(0.07), p(0.98), r(0.04)
App ID 7 ROUGE scores:
rouge-1: f(0.07), p(0.92), r(0.04)
rouge-2: f(0.03), p(0.68), r(0.01)
rouge-l: f(0.07), p(0.92), r(0.04)
App ID 8 ROUGE scores:
rouge-1: f(0.34), p(0.95), r(0.20)
rouge-2: f(0.19), p(0.72), r(0.11)
rouge-l: f(

- BERT SCORES

In [8]:
!pip install bert-score

In [9]:
from bert_score import score

candidates = generated_data_texts
references = privacy_policy_texts

P, R, F1 = score(candidates, references, lang='en', verbose=True)

for app_id, p, r, f1 in zip([data[0] for data in data1], P, R, F1):
    print(f"App ID {app_id}: Precision = {p:.4f}, Recall = {r:.4f}, F1 Score = {f1:.4f}")

average_precision = P.mean().item()
average_recall = R.mean().item()
average_f1 = F1.mean().item()

print(f"Average BERTScore Precision: {average_precision:.4f}")
print(f"Average BERTScore Recall: {average_recall:.4f}")
print(f"Average BERTScore F1: {average_f1:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 281.63 seconds, 0.17 sentences/sec
App ID 1: Precision = 0.8133, Recall = 0.7992, F1 Score = 0.8062
App ID 2: Precision = 0.8643, Recall = 0.7988, F1 Score = 0.8303
App ID 3: Precision = 0.8712, Recall = 0.8070, F1 Score = 0.8379
App ID 4: Precision = 0.8307, Recall = 0.8276, F1 Score = 0.8291
App ID 5: Precision = 0.8360, Recall = 0.8306, F1 Score = 0.8333
App ID 6: Precision = 0.8870, Recall = 0.7708, F1 Score = 0.8248
App ID 7: Precision = 0.8888, Recall = 0.8244, F1 Score = 0.8554
App ID 8: Precision = 0.8855, Recall = 0.8388, F1 Score = 0.8615
App ID 9: Precision = 0.8707, Recall = 0.7974, F1 Score = 0.8324
App ID 10: Precision = 0.7971, Recall = 0.7835, F1 Score = 0.7902
App ID 11: Precision = 0.9061, Recall = 0.8072, F1 Score = 0.8538
App ID 12: Precision = 0.7784, Recall = 0.7805, F1 Score = 0.7794
App ID 13: Precision = 0.9252, Recall = 0.8256, F1 Score = 0.8725
App ID 14: Precision = 0.8534, Recall = 0.7884, F1 Score = 0.8196
App ID 15: Precision = 0.8683, Recall = 0.